In [1]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mglearn
from sklearn import svm
import datetime as dt
from sklearn.ensemble import RandomForestClassifier 

In [2]:
df = pd.read_csv("./match_data_yearly/all_years.csv")
df["Date"] = pd.to_datetime(df["Date"])
df = df.set_index(df["Date"])

In [4]:
year = 2020

df_past = pd.concat([df[f"{year-3}"],df[f"{year-2}"],df[f"{year-1}"]])
X_train=df_past[["HomeElo","AwayElo"]].values
y_train = df_past["W/L"].values

df_future = df[f"{year}"]
X_test = df_future[["HomeElo","AwayElo"]].values
y_test = df_future["W/L"].values


rf= RandomForestClassifier(max_depth=5)  
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
y_pred_proba=rf.predict_proba(X_test)

accuracy = rf.score(X_test,y_test) 
# accuracies.append(accuracy)


count=0
pred_winrate_sum=0 # 予測勝率の和
actual_winrate_sum=0 # 実勝率の和

for index,row in df_future.iterrows():
    if row["HomeElo"] > row["AwayElo"]: # ホームが実力上位
        pred_winrate_sum += y_pred_proba[count][1] # ホームの予想勝率を加算
        if y_test[count] == 1: # 実際にホームが勝ったならば､1を加算
            actual_winrate_sum += 1
    elif row["AwayElo"] > row["HomeElo"]: # アウェイが実力上位
        pred_winrate_sum += y_pred_proba[count][2] # アウェイの予想勝率を加算
        if y_test[count] == 2: # 実際にアウェイが勝ったならば､1を加算
             actual_winrate_sum += 1 

    count+= 1

calibration = pred_winrate_sum/actual_winrate_sum
# calibrations.append(calibration)

In [6]:
accuracy

0.4215686274509804

In [7]:
calibration

0.8260637025025008

### BEGIN

In [11]:
from get_recent_match_id import get_recent_match_id

In [12]:
df["2020"]

,ID,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,Stadium,Attendances,HomeElo,AwayElo,HomeED,AwayED
Date,,,,,,,,,,,,,,,,,
2020-02-21,20011203,2020,1,2020-02-21,shonan-bellmare,urawa-red-diamonds,12,3,2,3,2,ＢＭＷス,13071,1422,1484,0,0
2020-02-22,20015408,2020,1,2020-02-22,vegalta-sendai,nagoya-grampus-eight,54,8,1,1,0,ユアスタ,13968,1459,1431,0,0
2020-02-22,20012031,2020,1,2020-02-22,cerezo-osaka,oita-trinita,20,31,1,0,1,ヤンマー,15535,1558,1427,0,0
2020-02-22,20011114,2020,1,2020-02-22,kashiwa-reysol,consadole-sapporo,11,14,4,2,1,三協Ｆ柏,12468,1503,1481,0,0
2020-02-22,20012133,2020,1,2020-02-22,kawasaki-frontale,sagan-tosu,21,33,0,0,0,等々力,21117,1628,1456,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-19,20340120,2020,34,2020-12-19,kashima-antlers,cerezo-osaka,1,20,1,1,0,カシマ,11251,1594,1560,4,-6
2020-12-19,20340810,2020,34,2020-12-19,nagoya-grampus-eight,sanfrecce-hiroshima,8,10,1,0,1,豊田ス,16825,1515,1524,9,-13
2020-12-19,20343331,2020,34,2020-12-19,sagan-tosu,oita-trinita,33,31,2,2,0,駅スタ,8966,1453,1458,17,7


In [70]:
match_id = 20340907
home = "gamba-osaka"
away = "shimizu-s-pulse"

home_recent_stats = pd.DataFrame()
away_recent_stats = pd.DataFrame()

# Home
for mid in get_recent_match_id(match_id)[0]:
    s = pd.read_csv(f"./stats/2020/{mid}.csv", index_col=0)[home]
    home_recent_stats = pd.concat([home_recent_stats, s], axis = 1)
    
# Away
for mid in get_recent_match_id(match_id)[1]:
    s = pd.read_csv(f"./stats/2020/{mid}.csv", index_col=0)[away]
    away_recent_stats = pd.concat([away_recent_stats, s], axis = 1)
    
    

In [71]:
home_recent_stats

,gamba-osaka,gamba-osaka,gamba-osaka,gamba-osaka,gamba-osaka
ExpG,2.138,0.932,2.814,0.475,1.61
Shots,13.000,12.000,20.000,6.000,13.00
ShotsOnTarget,9.000,5.000,7.000,1.000,7.00
ShotsFromPK,0.000,0.000,0.000,0.000,0.00
Passes,364.000,320.000,602.000,562.000,577.00
Crosses,5.000,9.000,29.000,11.000,15.00
DirectFK,9.000,6.000,12.000,7.000,5.00
IndirectFK,3.000,4.000,8.000,2.000,2.00
CK,6.000,0.000,11.000,5.000,8.00
Throwin,21.000,27.000,14.000,21.000,10.00


In [72]:
away_recent_stats

,shimizu-s-pulse,shimizu-s-pulse,shimizu-s-pulse,shimizu-s-pulse,shimizu-s-pulse
ExpG,0.621,1.004,1.306,1.83,0.838
Shots,9.000,11.000,15.000,15.00,11.000
ShotsOnTarget,3.000,0.000,8.000,4.00,5.000
ShotsFromPK,0.000,0.000,0.000,0.00,0.000
Passes,697.000,553.000,394.000,627.00,381.000
Crosses,9.000,14.000,5.000,18.00,7.000
DirectFK,11.000,14.000,14.000,4.00,10.000
IndirectFK,1.000,2.000,2.000,0.00,2.000
CK,2.000,7.000,2.000,7.00,2.000
Throwin,21.000,20.000,20.000,24.00,22.000


In [73]:
home_ave_recent_stats = home_recent_stats.mean(axis=1)
home_ave_recent_stats

ExpG                       1.5938
Shots                     12.8000
ShotsOnTarget              5.8000
ShotsFromPK                0.0000
Passes                   485.0000
Crosses                   13.8000
DirectFK                   7.8000
IndirectFK                 3.8000
CK                         6.0000
Throwin                   18.6000
Dribbling                  9.2000
Tackles                   19.6000
Clearances                20.0000
Intercepts                 2.8000
Offsides                   1.8000
YellowCards                1.0000
RedCards                   0.0000
30mLineEntries            41.8000
PenaltyAreaEntries        12.8000
DistanceCovered       118485.4000
Sprints                  164.4000
Attacks                  119.2000
ChanceBuildingRate        10.6000
Possession                45.2000
dtype: float64

In [74]:
away_ave_recent_stats = away_recent_stats.mean(axis=1)
away_ave_recent_stats

ExpG                       1.1198
Shots                     12.2000
ShotsOnTarget              4.0000
ShotsFromPK                0.0000
Passes                   530.4000
Crosses                   10.6000
DirectFK                  10.6000
IndirectFK                 1.4000
CK                         4.0000
Throwin                   21.4000
Dribbling                 12.4000
Tackles                   19.8000
Clearances                25.6000
Intercepts                 2.4000
Offsides                   1.6000
YellowCards                1.0000
RedCards                   0.0000
30mLineEntries            34.2000
PenaltyAreaEntries        11.8000
DistanceCovered       118634.6000
Sprints                  175.4000
Attacks                  115.4000
ChanceBuildingRate        10.6400
Possession                49.6800
dtype: float64

In [81]:
try:
    home_ave_recent_stats.drop("ExpG", inplace=True)
    away_ave_recent_stats.drop("ExpG", inplace=True)
except KeyError:
    pass

try:
    home_ave_recent_stats.drop("Sprints", inplace=True)
    away_ave_recent_stats.drop("Sprints", inplace=True)
except KeyError:
    pass

try:
    home_ave_recent_stats.drop("DistanceCovered", inplace=True)
    away_ave_recent_stats.drop("DistanceCovered", inplace=True)
except KeyError:
    pass

In [82]:
away_ave_recent_stats

Shots                  12.20
ShotsOnTarget           4.00
ShotsFromPK             0.00
Passes                530.40
Crosses                10.60
DirectFK               10.60
IndirectFK              1.40
CK                      4.00
Throwin                21.40
Dribbling              12.40
Tackles                19.80
Clearances             25.60
Intercepts              2.40
Offsides                1.60
YellowCards             1.00
RedCards                0.00
30mLineEntries         34.20
PenaltyAreaEntries     11.80
Attacks               115.40
ChanceBuildingRate     10.64
Possession             49.68
dtype: float64

In [83]:
away_ave_recent_stats

Shots                  12.20
ShotsOnTarget           4.00
ShotsFromPK             0.00
Passes                530.40
Crosses                10.60
DirectFK               10.60
IndirectFK              1.40
CK                      4.00
Throwin                21.40
Dribbling              12.40
Tackles                19.80
Clearances             25.60
Intercepts              2.40
Offsides                1.60
YellowCards             1.00
RedCards                0.00
30mLineEntries         34.20
PenaltyAreaEntries     11.80
Attacks               115.40
ChanceBuildingRate     10.64
Possession             49.68
dtype: float64

In [84]:

def get_ave_recent_stats(match_id):
    home = ""
    away = ""
    
    df = pd.read_csv("./match_data_yearly/all_years.csv")
    for index,row in df.iterrows():
        if row["ID"] == match_id:
            target_line = index 
            home = row["Home"]
            away = row["Away"]
            
    home_recent_stats = pd.DataFrame()
    away_recent_stats = pd.DataFrame()

    # Home
    for mid in get_recent_match_id(match_id)[0]:
        s = pd.read_csv(f"./stats/2020/{mid}.csv", index_col=0)[home]
        home_recent_stats = pd.concat([home_recent_stats, s], axis = 1)
    home_ave_recent_stats = home_recent_stats.mean(axis=1)
    home_ave_recent_stats

    # Away
    for mid in get_recent_match_id(match_id)[1]:
        s = pd.read_csv(f"./stats/2020/{mid}.csv", index_col=0)[away]
        away_recent_stats = pd.concat([away_recent_stats, s], axis = 1)     
    away_ave_recent_stats = away_recent_stats.mean(axis=1)
    away_ave_recent_stats
    
    try:
        home_ave_recent_stats.drop("ExpG", inplace=True)
        away_ave_recent_stats.drop("ExpG", inplace=True)
    except KeyError:
        pass

    try:
        home_ave_recent_stats.drop("Sprints", inplace=True)
        away_ave_recent_stats.drop("Sprints", inplace=True)
    except KeyError:
        pass

    try:
        home_ave_recent_stats.drop("DistanceCovered", inplace=True)
        away_ave_recent_stats.drop("DistanceCovered", inplace=True)
    except KeyError:
        pass

    return home_ave_recent_stats,away_ave_recent_stats

In [86]:
get_ave_recent_stats(20310721)

(Shots                  15.40
 ShotsOnTarget           5.00
 ShotsFromPK             0.00
 Passes                481.80
 Crosses                13.00
 DirectFK                8.60
 IndirectFK              1.20
 CK                      5.40
 Throwin                19.60
 Dribbling              11.00
 Tackles                18.20
 Clearances             22.80
 Intercepts              2.60
 Offsides                1.40
 YellowCards             0.40
 RedCards                0.00
 30mLineEntries         37.20
 PenaltyAreaEntries     12.60
 Attacks               110.80
 ChanceBuildingRate     13.88
 Possession             46.50
 dtype: float64,
 Shots                  17.20
 ShotsOnTarget           7.00
 ShotsFromPK             0.20
 Passes                548.60
 Crosses                14.20
 DirectFK               13.80
 IndirectFK              1.60
 CK                      8.40
 Throwin                23.80
 Dribbling              12.00
 Tackles                20.60
 Clearances            